In [1]:
!pip install rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 44.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


**calculate IoU**

In [3]:
import rasterio
import numpy as np
from rasterio.enums import Resampling

def resample_to_match(source_path, match_path, output_path):
    """Resample source raster to match the dimensions and transform of match_path."""
    with rasterio.open(match_path) as match:
        # Get the metadata and dimensions of the benchmark map
        match_meta = match.meta.copy()

        with rasterio.open(source_path) as src:
            # Update metadata for the output
            match_meta.update({
                'dtype': src.dtypes[0],
                'nodata': 0,
                'width': match.width,
                'height': match.height,
                'transform': match.transform
            })

            # Resample source to match the dimensions of benchmark map
            data = src.read(
                out_shape=(src.count, match.height, match.width),
                resampling=Resampling.nearest
            )

            # Save the resampled data to output path
            with rasterio.open(output_path, 'w', **match_meta) as dest:
                dest.write(data)

def calculate_iou(predicted_map_path, benchmark_map_path):
    """Calculate Intersection over Union (IoU) between predicted and benchmark maps."""
    with rasterio.open(predicted_map_path) as pred_src, rasterio.open(benchmark_map_path) as bench_src:
        predicted = pred_src.read(1)
        benchmark = bench_src.read(1)

        # Calculate Intersection and Union
        intersection = np.logical_and(predicted == 1, benchmark == 1).sum()
        union = np.logical_or(predicted == 1, benchmark == 1).sum()

        # Calculate IoU
        iou = intersection / union if union != 0 else 0
        return iou

# File paths to the flood maps
sentinel1_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_VH_S1_Resample.tif'
sentinel2_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_NDWI_S2_Resample.tif'
landsat_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_NDWI_LSAT_Resample.tif'
benchmark_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/waterExtent_arcgis0_1.tif'

# Resample predicted maps to match benchmark dimensions
resample_to_match(sentinel1_map_path, benchmark_map_path, 'resampled_sentinel1_map.tif')
resample_to_match(sentinel2_map_path, benchmark_map_path, 'resampled_sentinel2_map.tif')
resample_to_match(landsat_map_path, benchmark_map_path, 'resampled_landsat_map.tif')

# Calculate IoU using resampled files
iou_sentinel1 = calculate_iou('resampled_sentinel1_map.tif', benchmark_map_path)
iou_sentinel2 = calculate_iou('resampled_sentinel2_map.tif', benchmark_map_path)
iou_landsat = calculate_iou('resampled_landsat_map.tif', benchmark_map_path)

print("IoU for Sentinel-1:", iou_sentinel1)
print("IoU for Sentinel-2:", iou_sentinel2)
print("IoU for Landsat:", iou_landsat)


IoU for Sentinel-1: 0.21806214169438742
IoU for Sentinel-2: 0.4376902339333855
IoU for Landsat: 0.3440952130457654


**Overlap Percentage with ArcGis**

In [35]:
intersected_Area_s1= 3081; #from .tif in Arcgis;
intersected_Area_s2=5033; #from .tif in Arcgis;
intersected_Area_ls=5315; #from .tif in Arcgis ;
Flood_area_arc=8790;

Overlap_Percentage_s1=(intersected_Area_s1/Flood_area_arc)*100;
Overlap_Percentage_s2=(intersected_Area_s2/Flood_area_arc)*100;
Overlap_Percentage_ls=(intersected_Area_ls/Flood_area_arc)*100;

print("Overlap Percentage sentinel-1:",Overlap_Percentage_s1);
print("Overlap Percentage sentinel-2:",Overlap_Percentage_s2);
print("Overlap Percentage landsat:",Overlap_Percentage_ls);

Overlap Percentage sentinel-1: 35.051194539249146
Overlap Percentage sentinel-2: 57.25824800910125
Overlap Percentage landsat: 60.46643913538111
